In [ ]:
import sys
import random

import mne
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torch.nn as nn
import os
import math
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
import importlib
from einops import rearrange, reduce, repeat
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
%matplotlib inline

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# ------------------------------ bci competition dataset ------------------------------
from moabb.datasets import BNCI2014001
from moabb.paradigms import MotorImagery
# fmin = 4
# fmax = 100
fmin = 5
fmax = 60
tmin = 0
tmax = None

X_list = []
y_list = []

# Load the dataset
subjects = [1, 2, 3, 4, 5, 6, 7, 8, 9]
for subject in subjects:
    dataset = BNCI2014001()
    events = ['left_hand', 'right_hand', 'feet', 'tongue']
    labels_to_int = {'left_hand':0, 'right_hand':1, 'feet':2, 'tongue':3}
    paradigm = MotorImagery(
        events=events, n_classes=len(events), fmin=fmin, fmax=fmax, tmin=tmin, tmax=tmax
    )

    data, labels, _ = paradigm.get_data(dataset=dataset, subjects=[subject])
    data = data[:, :, :500]

    labels = [labels_to_int[label] for label in labels]

    X_list.append(torch.tensor(data).to(dtype=torch.float32, device=device))
    y_list.append(torch.tensor(labels).to(dtype=torch.long, device=device))

In [ ]:
from Models.FinalModels import Eegnet, EegnetTemporal, EegnetSpatial, EegnetTemporalSpatial

#tests info
channels = 22
classes = 4
samples=500
frequency=250
kernel_length = frequency//2
seed = 1330
models = [
    'eegnet',
    'eegnet + temporal',
    'eegnet + spatial',
    'eegnet + spatial + temporal'
]

def getModel(name):
    if name == 'eegnet':
        return Eegnet(n_times=samples, n_channels=channels, n_classes=classes, kernel_length=kernel_length)

    if name == 'eegnet + temporal':
        return EegnetTemporal(n_times=samples, n_channels=channels, n_classes=classes, kernel_length=kernel_length)

    if name == 'eegnet + spatial':
        return EegnetSpatial(n_times=samples, n_channels=channels, n_classes=classes, kernel_length=kernel_length)

    if name == 'eegnet + spatial + temporal':
        return EegnetTemporalSpatial(n_times=samples, n_channels=channels, n_classes=classes, kernel_length=kernel_length)


In [ ]:
# ------------------------------ bci competition dataset ------------------------------
from moabb.datasets import BNCI2014001
from moabb.paradigms import MotorImagery
# fmin = 4
# fmax = 100
fmin = 5
fmax = 60
tmin = 0
tmax = None

X_list = []
y_list = []

# Load the dataset
subjects = [1]
dataset = BNCI2014001()
events = ['left_hand', 'right_hand', 'feet', 'tongue']
labels_to_int = {'left_hand':0, 'right_hand':1, 'feet':2, 'tongue':3}
paradigm = MotorImagery(
    events=events, n_classes=len(events), fmin=fmin, fmax=fmax, tmin=tmin, tmax=tmax
)

data, labels, _ = paradigm.get_data(dataset=dataset, subjects=subjects)
data = data[:, :, :500]

labels = [labels_to_int[label] for label in labels]

X = torch.tensor(data).to(dtype=torch.float32, device=device)
y = torch.tensor(labels).to(dtype=torch.long, device=device)

In [ ]:
from Models.FinalModels import exists
import torch
import torch.nn.functional as F
import torch.nn as nn
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange
from x_transformers import Encoder

from utils.utils import initialize_weight
from base.layers import Conv2dWithConstraint


class EegnetModified(nn.Module):
    def __init__(
            self,
            n_channels,
            kernel_length,
            F1,
            D,
            F2,
            pool1_stride,
            pool2_stride,
            dropout_rate,
            weight_init_method=None
    ):
        super().__init__()

        # Spectral
        self.spectral = nn.Sequential(
            Rearrange('b c w -> b 1 c w'),
            nn.Conv2d(1, F1, (1, kernel_length), bias=False, padding='same'),
            nn.BatchNorm2d(F1),
        )

        #residual spectral
        self.residual_spc =  nn.Sequential(
            Rearrange('b c w -> b 1 c w'),
        )

        # Spatial
        self.spatial = nn.Sequential(
            Conv2dWithConstraint(F1, F1 * D, (n_channels, 1), bias=False, groups=F1),
            nn.BatchNorm2d(F1 * D),
            nn.ELU(),
            nn.AvgPool2d((1, pool1_stride)),
            nn.Dropout(dropout_rate),
        )

        #residual Spatial
        self.residual_spt = nn.Sequential(
            nn.Conv2d(F1, F1 * D, (n_channels, 1), bias=False, groups=F1),
            nn.ELU(),
            nn.AvgPool2d((1, pool1_stride)),
            nn.Dropout(dropout_rate),
        )

        # Temporal
        self.temporal = nn.Sequential(
            nn.Conv2d(F1 * D, F2, (1, n_channels), bias=False, padding='same', groups=F2),
            nn.Conv2d(F2, F2, 1, bias=False),
            nn.BatchNorm2d(F2),
            nn.ELU(),
            nn.AvgPool2d((1, pool2_stride)),
            nn.Dropout(dropout_rate),
        )

        initialize_weight(self, weight_init_method)

    def forward(self, x):
        #spectral
        spc_out = self.spectral(x)
        spc_residual = self.residual_spc(x)

        # Spatial
        spt_out = self.spatial(spc_out + spc_residual)
        spt_residual = self.residual_spt(spc_out + spc_residual)

        # Temporal
        out = self.temporal(spt_out + spt_residual)

        return out

class AttnBlock(nn.Module):
    def __init__(self, dim, nheads, dropout=0.5):
        super().__init__()
        self.ffd1 = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(in_features=dim, out_features=dim),
            nn.ELU(),
            nn.Dropout(dropout)
        )

        self.attention1 = nn.MultiheadAttention(embed_dim=dim, num_heads=nheads, batch_first=True, dropout=dropout)

        self.ff2 = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(in_features=dim, out_features=dim),
            nn.ELU(),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        b1_out = self.ffd1(x)
        at1_out, attn_output_weights = self.attention1(b1_out, b1_out ,b1_out)
        ffd1_out = self.ff2(at1_out + x)
        return ffd1_out



class SelfAttentionConv(nn.Module):
    def __init__(self, n_channels):
        super().__init__()
        self.query,self.key,self.value = [self._conv(n_channels, c) for c in (n_channels//8,n_channels//8,n_channels)]
        self.gamma = nn.Parameter(torch.tensor([0.]))

    def _conv(self,n_in,n_out):
        return nn.Conv1d(n_in, n_out, kernel_size=1, bias=False)

    def forward(self, x):
        #Notation from the paper.
        size = x.size()
        f,g,h = self.query(x), self.key(x), self.value(x)
        beta = F.softmax(f.transpose(1,2) @ g, dim=1)
        o = self.gamma * (h @ beta) + x
        return o.view(*size).contiguous()


In [ ]:
from Models.FinalModels import FeatureExtraction
from x_transformers import Encoder, ContinuousTransformerWrapper


class Temporal(nn.Module):
    def __init__(
            self,
            n_times,
            n_classes,
            n_channels,
            kernel_length,
            dropout_rate=0.5,
            F1=8,
            F2=16,
            D=2,
            pool1_stride=16,
            pool2_stride=8,
    ):
        super().__init__()

        self.feature_extraction_output = F2 * (
                (((n_times - pool1_stride) // pool1_stride + 1) - pool2_stride) // pool2_stride + 1)

        self.feature_extraction = nn.Sequential(
            EegnetModified(n_channels=n_channels, kernel_length=kernel_length, F1=F1, D=D, F2=F2,
                           pool1_stride=pool1_stride, pool2_stride=pool2_stride, dropout_rate=dropout_rate),
            nn.Flatten(),
        )

        self.head = nn.Sequential(
            nn.Linear(in_features=self.feature_extraction_output, out_features=n_classes)
        )

    def forward(self, x, targets):
        fe_out = self.feature_extraction(x)
        logits = self.head(fe_out)

        if targets is None:
            loss = None
        else:
            loss = F.cross_entropy(logits, targets)

        return logits, loss, {}

In [ ]:
model = Temporal(n_times=samples, n_channels=channels, n_classes=classes, kernel_length=kernel_length)
model = model.to(device=device)

In [ ]:
#params evaluation
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, np.prod(param.size()))

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print('total: ', params)

In [ ]:
from modules.TrainTester import TrainerTester

skf = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)

for train_index, test_index in skf.split(X, y):
    optimizer = torch.optim.AdamW(model.parameters())

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    TrainerTester.train_loop(model, optimizer, X_train, y_train, X_test, y_test, 1, [], batch_size=32, iterations=2000)
    loss, accuracy, kappa = TrainerTester.test_loss(model, X_test, y_test)
    print(loss.item(), accuracy, kappa)
    break



In [ ]:
# from modules.TrainTester import TrainerTester
# import pandas as pd
#
#
# dataframe_dict = {}
# test = 0
# for model_name in models:
#     print(f"iniciando treinamento da model {model_name}")
#     subject_counter = 1
#     for X, y in zip(X_list, y_list):
#         print(f"iniciando teste de sujeito {subject_counter}")
#
#         dataframe_dict[f"{model_name} sujeito {subject_counter} kappa"] = []
#         dataframe_dict[f"{model_name} sujeito {subject_counter} acc"] = []
#         dataframe_dict[f"{model_name} sujeito {subject_counter} loss"] = []
#         for i in range(0, 5):
#             print(f'iniciando 5k fold {i+1}/5')
#
#             skf = StratifiedKFold(n_splits=5, random_state=seed+i, shuffle=True)
#             k_fold_counter = 0
#             for train_index, test_index in skf.split(X, y):
#                 model = getModel(model_name)
#                 model = model.to(device=device)
#                 optimizer = torch.optim.AdamW(model.parameters(), lr=8e-4)
#
#                 X_train, X_test = X[train_index], X[test_index]
#                 y_train, y_test = y[train_index], y[test_index]
#
#                 TrainerTester.simplified_train_loop(model, optimizer, X_train, y_train, X_test, y_test, batch_size=32, iterations=2000)
#
#                 loss, accuracy, kappa = TrainerTester.test_loss(model, X_test, y_test)
#                 print(f'{model_name} results in 5k fold {k_fold_counter}/5\n kappa: {kappa}\n accuracy: {accuracy}\n loss: {loss}')
#                 k_fold_counter += 1
#
#                 dataframe_dict[f"{model_name} sujeito {subject_counter} kappa"].append(kappa)
#                 dataframe_dict[f"{model_name} sujeito {subject_counter} acc"].append(accuracy)
#                 dataframe_dict[f"{model_name} sujeito {subject_counter} loss"].append(loss)
#
#         subject_counter += 1
#         final_dataframe = pd.DataFrame.from_dict(dataframe_dict)
#         final_dataframe.to_csv('resultados_paper.csv')
#
# final_dataframe = pd.DataFrame.from_dict(dataframe_dict)
# final_dataframe.to_csv('resultados_paper.csv')